# HR Analytics

<img src = 'https://datahack-prod.s3.ap-south-1.amazonaws.com/__sized__/contest_cover/hr_1920x480_s5WuoZs-thumbnail-1200x1200-90.jpg'>

Practice Problem: https://datahack.analyticsvidhya.com/contest/wns-analytics-hackathon-2018-1/

## HR Analytics

HR analytics is revolutionising the way human resources departments operate, leading to higher efficiency and better results overall. Human resources has been using analytics for years. However, the collection, processing and analysis of data has been largely manual, and given the nature of human resources dynamics and HR KPIs, the approach has been constraining HR. Therefore, it is surprising that HR departments woke up to the utility of machine learning so late in the game. Here is an opportunity to try predictive analytics in identifying the employees most likely to get promoted.

## Problem Statement

Your client is a large MNC and they have 9 broad verticals across the organisation. One of the problem your client is facing is around identifying the right people for promotion *(only for manager position and below)* and prepare them in time. Currently the process, they are following is:

* They first identify a set of employees based on recommendations/ past performance
* Selected employees go through the separate training and evaluation program for each vertical. These programs are based on the required skill of each vertical
* At the end of the program, based on various factors such as training performance, KPI completion (only employees with KPIs completed greater than 60% are considered) etc., employee gets promotion

For above mentioned process, the final promotions are only announced after the evaluation and this leads to delay in transition to their new roles. Hence, company needs your help in identifying the eligible candidates at a particular checkpoint so that they can expedite the entire promotion cycle. 

<img src = 'https://s3-ap-south-1.amazonaws.com/av-blog-media/wp-content/uploads/2018/09/wns_hack_im_1.jpg'>

They have provided multiple attributes around Employee's past and current performance along with demographics. Now, The task is to predict whether a potential promotee at checkpoint in the test set will be promoted or not after the evaluation process.

## Evaluation Metric

The evaluation metric for this competition is F1 Score.

## Public and Private Split

Test data is further randomly divided into Public (40%) and Private (60%) data.

Your initial responses will be checked and scored on the Public data.
The final rankings would be based on your private score which will be published once the competition is over.

## Entorno

In [1]:
import sys
sys.version

'3.7.9 (default, Aug 31 2020, 17:10:11) [MSC v.1916 64 bit (AMD64)]'

In [2]:
!conda info --envs

# conda environments:
#
base                  *  C:\Users\antho\Anaconda3



## Paquetes

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import os
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm, tqdm_notebook
from pathlib import Path
import random
import warnings
import pickle

warnings.filterwarnings('ignore')


seed = 2020
random.seed(seed)

pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 400)
sns.set()

DATA = Path('../../data') 
RAW  = DATA/'raw'
PROCESSED = DATA/'processed'
SUBMISSIONS = DATA/'submissions'    

MODEL = Path('../../model') 

In [4]:
pd.__version__

'1.1.3'

In [5]:
np.__version__

'1.19.2'

In [6]:
sklearn.__version__

'0.23.2'

In [7]:
id_columns = 'employee_id'
target = 'is_promoted'

## Lectura de datos

In [8]:
os.listdir(f'{PROCESSED}')

['.DS_Store',
 'preprocess_v1_capping_values.pkl',
 'preprocess_v1_impute_values.pkl',
 'preprocess_v1_ohe.pkl',
 'preprocess_v1_ohe_columns.pkl',
 'preprocess_v1_over50_train.csv',
 'preprocess_v1_scaler.pkl',
 'preprocess_v1_smote20_train.csv',
 'preprocess_v1_smote50_train.csv',
 'preprocess_v1_smoteTomek20_train.csv',
 'preprocess_v1_smoteTomek50_train.csv',
 'preprocess_v1_train.csv',
 'preprocess_v1_under50_train.csv',
 'preprocess_v1_val.csv',
 'preprocess_v2_capping_values.pkl',
 'preprocess_v2_knnimputation.pkl',
 'preprocess_v2_ohe.pkl',
 'preprocess_v2_ohe_columns.pkl',
 'preprocess_v2_over50_train.csv',
 'preprocess_v2_scaler.pkl',
 'preprocess_v2_scalerimputation.pkl',
 'preprocess_v2_smote20_train.csv',
 'preprocess_v2_smote50_train.csv',
 'preprocess_v2_smoteTomek20_train.csv',
 'preprocess_v2_smoteTomek50_train.csv',
 'preprocess_v2_test.csv',
 'preprocess_v2_train.csv',
 'preprocess_v2_under50_train.csv',
 'preprocess_v2_val.csv']

## Entrenamiento V1 sin balanceo

In [9]:
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import precision_recall_curve, roc_auc_score, f1_score

In [10]:
preproc_train = [file for file in os.listdir(f'{PROCESSED}') if file.endswith('train.csv')]
preproc_train

['preprocess_v1_over50_train.csv',
 'preprocess_v1_smote20_train.csv',
 'preprocess_v1_smote50_train.csv',
 'preprocess_v1_smoteTomek20_train.csv',
 'preprocess_v1_smoteTomek50_train.csv',
 'preprocess_v1_train.csv',
 'preprocess_v1_under50_train.csv',
 'preprocess_v2_over50_train.csv',
 'preprocess_v2_smote20_train.csv',
 'preprocess_v2_smote50_train.csv',
 'preprocess_v2_smoteTomek20_train.csv',
 'preprocess_v2_smoteTomek50_train.csv',
 'preprocess_v2_train.csv',
 'preprocess_v2_under50_train.csv']

In [11]:
preproc_val = [file for file in os.listdir(f'{PROCESSED}') if file.endswith('val.csv')]
preproc_val

['preprocess_v1_val.csv', 'preprocess_v2_val.csv']

In [12]:
for train_file in sorted(preproc_train):
    df_train = pd.read_csv(f'{PROCESSED}/{train_file}', compression = 'zip')
    df_val = pd.read_csv(f'{PROCESSED}/{preproc_val[0]}', compression = 'zip')
    
    print(f'label: {train_file:35} \tnrows: {len(df_train)} \t%target train: {df_train[target].mean():.4f} \t%target val: {df_val[target].mean():.4f}')

label: preprocess_v1_over50_train.csv      	nrows: 80224 	%target train: 0.5000 	%target val: 0.0852
label: preprocess_v1_smote20_train.csv     	nrows: 48134 	%target train: 0.1667 	%target val: 0.0852
label: preprocess_v1_smote50_train.csv     	nrows: 80224 	%target train: 0.5000 	%target val: 0.0852
label: preprocess_v1_smoteTomek20_train.csv 	nrows: 46412 	%target train: 0.1543 	%target val: 0.0852
label: preprocess_v1_smoteTomek50_train.csv 	nrows: 79638 	%target train: 0.5000 	%target val: 0.0852
label: preprocess_v1_train.csv             	nrows: 43846 	%target train: 0.0852 	%target val: 0.0852
label: preprocess_v1_under50_train.csv     	nrows: 7468 	%target train: 0.5000 	%target val: 0.0852
label: preprocess_v2_over50_train.csv      	nrows: 80224 	%target train: 0.5000 	%target val: 0.0852
label: preprocess_v2_smote20_train.csv     	nrows: 48134 	%target train: 0.1667 	%target val: 0.0852
label: preprocess_v2_smote50_train.csv     	nrows: 80224 	%target train: 0.5000 	%target v

In [13]:
train_file = 'preprocess_v1_train.csv'
val_file = 'preprocess_v1_val.csv'


In [14]:
from sklearn.model_selection import ParameterGrid
from sklearn.tree import DecisionTreeClassifier

In [16]:
cv_params = {'criterion': ['gini', 'entropy'],
             'min_samples_split': [50, 100, 250, 500, 1000, 200],
             'max_depth': [10, 20, 30, 40, 50],
            'random_state': [seed]}

params_grid = list(ParameterGrid(cv_params))

In [17]:
df_results = pd.DataFrame(columns = ['preproc_label', 'model_label', 'método', 'parámetros', 'columnas_out',
                                     'auc_train', 'auc_val', 'threshold','f1_train', 'f1_val'])


for xgb_params in tqdm(params_grid):
    
    for train_file in sorted(preproc_train):

        preproc_label = train_file.split('_train')[0]

        print('----------------------------------------------------------------------')
        print(xgb_params)
        print(train_file)
        print('----------------------------------------------------------------------')

        df_train = pd.read_csv(f'{PROCESSED}/{train_file}', compression = 'zip')
        df_val = pd.read_csv(f'{PROCESSED}/{preproc_val[0]}', compression = 'zip')

        X_train, y_train = df_train.drop(target, axis = 1), df_train[target]
        X_val, y_val = df_val.drop(target, axis = 1), df_val[target]

        tree = DecisionTreeClassifier(criterion = xgb_params["criterion"],max_depth =xgb_params["max_depth"], min_samples_split =xgb_params["min_samples_split"], random_state= xgb_params["random_state"])
        tree_fit = tree.fit( X_train,y_train )
                        
        
        #xgb_params_export = xgb_params.copy()
        #xgb_params_export.update(logi_fit.attributes())

        probs_train = tree_fit.predict(X_train)
        probs_val = tree_fit.predict(X_val)

        auc_train = roc_auc_score(y_train, probs_train)
        auc_val = roc_auc_score(y_val, probs_val)

        #best threshold
        prec, recall, threshold = precision_recall_curve(y_train, probs_train)
        prec_recall = pd.DataFrame({'prec': prec[:-1], 'recall': recall[:-1], 'threshold': threshold})
        prec_recall['f1'] = 2*prec_recall['prec']*prec_recall['recall'] / (prec_recall['prec'] + prec_recall['recall'])
        prec_recall = prec_recall.sort_values(by = 'f1', ascending = False).head(1)

        #f1 scores
        best_threshold = prec_recall['threshold'].values[0]
        f1_train = prec_recall['f1'].values[0]

        labels_val = np.where(probs_val >= best_threshold, 1, 0)
        f1_val = f1_score(y_val, labels_val)

        print(f'auc_train: {auc_train:.6f} \tauc_val: {auc_val:.6f} \tf1_train: {f1_train:.6f} \tf1_val: {f1_val:.6f}')

        results = [preproc_label, 'DecisionTreeClassifier', 'fit', xgb_params, '',
                  auc_train, auc_val, best_threshold, f1_train, f1_val]


        df_results.loc[len(df_results)] = results

  0%|          | 0/60 [00:00<?, ?it/s]

----------------------------------------------------------------------
{'criterion': 'gini', 'max_depth': 10, 'min_samples_split': 50, 'random_state': 2020}
preprocess_v1_over50_train.csv
----------------------------------------------------------------------
auc_train: 0.834289 	auc_val: 0.804918 	f1_train: 0.852623 	f1_val: 0.356841
----------------------------------------------------------------------
{'criterion': 'gini', 'max_depth': 10, 'min_samples_split': 50, 'random_state': 2020}
preprocess_v1_smote20_train.csv
----------------------------------------------------------------------
auc_train: 0.744316 	auc_val: 0.654845 	f1_train: 0.651182 	f1_val: 0.459734
----------------------------------------------------------------------
{'criterion': 'gini', 'max_depth': 10, 'min_samples_split': 50, 'random_state': 2020}
preprocess_v1_smote50_train.csv
----------------------------------------------------------------------
auc_train: 0.875848 	auc_val: 0.753349 	f1_train: 0.879783 	f1_val:

  2%|▏         | 1/60 [00:18<18:00, 18.31s/it]

auc_train: 0.826861 	auc_val: 0.808335 	f1_train: 0.840979 	f1_val: 0.363049
----------------------------------------------------------------------
{'criterion': 'gini', 'max_depth': 10, 'min_samples_split': 100, 'random_state': 2020}
preprocess_v1_over50_train.csv
----------------------------------------------------------------------
auc_train: 0.832344 	auc_val: 0.806747 	f1_train: 0.850864 	f1_val: 0.357293
----------------------------------------------------------------------
{'criterion': 'gini', 'max_depth': 10, 'min_samples_split': 100, 'random_state': 2020}
preprocess_v1_smote20_train.csv
----------------------------------------------------------------------
auc_train: 0.744228 	auc_val: 0.656487 	f1_train: 0.650025 	f1_val: 0.460944
----------------------------------------------------------------------
{'criterion': 'gini', 'max_depth': 10, 'min_samples_split': 100, 'random_state': 2020}
preprocess_v1_smote50_train.csv
----------------------------------------------------------

  3%|▎         | 2/60 [00:36<17:42, 18.32s/it]

...

auc_train: 0.803160 	auc_val: 0.800318 	f1_train: 0.821342 	f1_val: 0.349587
----------------------------------------------------------------------
{'criterion': 'entropy', 'max_depth': 50, 'min_samples_split': 1000, 'random_state': 2020}
preprocess_v1_over50_train.csv
----------------------------------------------------------------------
auc_train: 0.834763 	auc_val: 0.807971 	f1_train: 0.847292 	f1_val: 0.379925
----------------------------------------------------------------------
{'criterion': 'entropy', 'max_depth': 50, 'min_samples_split': 1000, 'random_state': 2020}
preprocess_v1_smote20_train.csv
----------------------------------------------------------------------
auc_train: 0.737522 	auc_val: 0.638163 	f1_train: 0.640033 	f1_val: 0.423263
----------------------------------------------------------------------
{'criterion': 'entropy', 'max_depth': 50, 'min_samples_split': 1000, 'random_state': 2020}
preprocess_v1_smote50_train.csv
----------------------------------------------

 98%|█████████▊| 59/60 [18:27<00:18, 18.49s/it]

auc_train: 0.789234 	auc_val: 0.787010 	f1_train: 0.813419 	f1_val: 0.327349
----------------------------------------------------------------------
{'criterion': 'entropy', 'max_depth': 50, 'min_samples_split': 200, 'random_state': 2020}
preprocess_v1_over50_train.csv
----------------------------------------------------------------------
auc_train: 0.877294 	auc_val: 0.775319 	f1_train: 0.882745 	f1_val: 0.394609
----------------------------------------------------------------------
{'criterion': 'entropy', 'max_depth': 50, 'min_samples_split': 200, 'random_state': 2020}
preprocess_v1_smote20_train.csv
----------------------------------------------------------------------
auc_train: 0.789816 	auc_val: 0.681023 	f1_train: 0.722065 	f1_val: 0.504360
----------------------------------------------------------------------
{'criterion': 'entropy', 'max_depth': 50, 'min_samples_split': 200, 'random_state': 2020}
preprocess_v1_smote50_train.csv
-------------------------------------------------

100%|██████████| 60/60 [18:46<00:00, 18.78s/it]

auc_train: 0.827665 	auc_val: 0.806942 	f1_train: 0.838093 	f1_val: 0.372558


In [18]:
df_results.head()

,preproc_label,model_label,método,parámetros,columnas_out,auc_train,auc_val,threshold,f1_train,f1_val
0,preprocess_v1_over50,DecisionTreeClassifier,fit,"{'criterion': 'gini', 'max_depth': 10, 'min_sa...",,0.834289,0.804918,1,0.852623,0.356841
1,preprocess_v1_smote20,DecisionTreeClassifier,fit,"{'criterion': 'gini', 'max_depth': 10, 'min_sa...",,0.744316,0.654845,1,0.651182,0.459734
2,preprocess_v1_smote50,DecisionTreeClassifier,fit,"{'criterion': 'gini', 'max_depth': 10, 'min_sa...",,0.875848,0.753349,1,0.879783,0.394545
3,preprocess_v1_smoteTomek20,DecisionTreeClassifier,fit,"{'criterion': 'gini', 'max_depth': 10, 'min_sa...",,0.791437,0.641052,1,0.733549,0.433578
4,preprocess_v1_smoteTomek50,DecisionTreeClassifier,fit,"{'criterion': 'gini', 'max_depth': 10, 'min_sa...",,0.885632,0.750183,1,0.884284,0.441074


In [20]:
MODELS = DATA/'models'

In [21]:
df_results.to_csv(f'{MODELS}/DecisionTreeClassifier.csv', index = False)

In [22]:
df_results.to_csv('DecisionTreeClassifier.csv')

In [23]:
df_results.to_excel(f'{MODELS}/DecisionTreeClassifier.xlsx', index = False)